In [ ]:
import pandas as pd
from twython import Twython
import pickle

with open("TwitterKeys.dat", "rb") as SaveFile:
    ReadData = pickle.load(file=SaveFile)
    
TwitterStream = Twython(ReadData["ConsumerKey"],ReadData["ConsumerSecret"])

In [ ]:
TwitterData = TwitterStream.search(q="data science -filter:retweets AND -filter:replies", max_id = 1220894112641114119, count=1000, tweet_mode = "extended")

In [ ]:
type(TwitterData)

In [ ]:
TwitterData.keys()

In [ ]:
TwitterData["statuses"][0]["full_text"]

In [ ]:
import urllib
print(TwitterData["search_metadata"])
print(TwitterData["search_metadata"]["next_results"])

urllib.parse.parse_qs((TwitterData["search_metadata"]["next_results"]).replace("?", "")).get("max_id")[0]

In [ ]:
import urllib
print(TwitterData["search_metadata"])
print(TwitterData["search_metadata"]["next_results"])

urllib.parse.parse_qs((TwitterData["search_metadata"]["next_results"]).replace("?", "")).get("max_id")[0]

In [ ]:
import numpy as np
np.iinfo("int64").max

In [ ]:
import pickle
with open("TwitterRawData.dat", "wb") as SaveFile:
    pickle.dump(TwitterData, file=SaveFile)

In [ ]:
with open("TwitterRawData.dat", "rb") as SaveFile:
    ReadData = pickle.load(file=SaveFile)

In [ ]:
print(ReadData["statuses"][0]["full_text"])

In [ ]:
len(TwitterData["statuses"])

# One function loading

In [1]:
def GetTweets(SearchTerms, MaxNumberOfTweets):
    import pandas as pd
    import numpy as np
    from twython import Twython
    import pickle
    import urllib
    
    # Get keys for Twitter Developer API
    with open("TwitterKeys.dat", "rb") as SaveFile:
        ReadData = pickle.load(file=SaveFile)

    twitterStream = Twython(ReadData["ConsumerKey"],ReadData["ConsumerSecret"])
    
    # Loop through stream until dry or max number of tweets reached
    twitterData = []
    max_id = int(np.iinfo(np.int64).max)

    maxQueryCount = int(MaxNumberOfTweets / 100) * 2  # Limit of 100 tweets per request. Arbitrarily query twitter at least 2x as often
    readTweetCount = 0
    
    for loopCount in range(maxQueryCount):
        # Pull data from stream
        streamData = twitterStream.search(
            q=str(["\"" + term + "\" " for term in SearchTerms]) + "-filter:retweets AND -filter:replies", # Querystring, removing potential duplicates
            max_id = max_id, # Get next set of tweets
            tweet_mode = "extended", # 280 char support instead of legacy 140 char
            count=MaxNumberOfTweets,
        )
        print("Loading ", str(len(streamData["statuses"])), " tweets.")
        # Save tweets
        twitterData.append(streamData["statuses"])
        readTweetCount += len(streamData["statuses"])

        # Get max_id and update marker
        max_id = streamData["search_metadata"].get("next_results")
        if(max_id):
            max_id = urllib.parse.parse_qs(max_id.replace("?", "")).get("max_id")
            if(max_id):
                max_id = int(max_id[0])
            else:
                break
        else:
            break
        
        # Check if got enough tweets
        if(readTweetCount >= MaxNumberOfTweets):
            break

    # Return flattened list of tweets (statuses)
    twitterData = pd.DataFrame([tweet for tweetGroup in twitterData for tweet in tweetGroup])

    print("")
    print("Dupes: ", str(twitterData["id"].duplicated().sum()))
    print("")
    display(twitterData.info())
    display(twitterData.head())
    
    return twitterData


In [2]:
TwitterData = GetTweets(["data science"], 1000)

# Save tweets to file
import pickle
with open("TwitterRawData.dat", "wb") as SaveFile:
    pickle.dump(TwitterData, file=SaveFile)

Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  97  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  100  tweets.
Loading  97  tweets.

Dupes:  0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 30 columns):
created_at                   1094 non-null object
id                           1094 non-null int64
id_str                       1094 non-null object
full_text                    1094 non-null object
truncated                    1094 non-null bool
display_text_range           1094 non-null object
entities                     1094 non-null object
metadata                     1094 non-null object
source                       1094 non-null object
in_reply_to_status_id        0 non-null object
in_reply_to_status_id_str    0 non-null object
in_reply_to_user_id          0 non-null object
in_reply_to_user_id_str      0 non-null object
in_rep

None

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status
0,Sat Jan 25 06:31:26 +0000 2020,1220957331149557765,1220957331149557765,i guess its time to switch majors. data scienc...,False,"[0, 64]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",None,...,1,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Sat Jan 25 06:23:40 +0000 2020,1220955374867701761,1220955374867701761,#TechnoCool: Data Science Community Rocked by ...,False,"[0, 192]","{'hashtags': [{'text': 'TechnoCool', 'indices'...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.russelladvisors.org"" rel=""...",None,...,1,0,False,False,en,False,NaN,NaN,NaN,NaN
2,Sat Jan 25 06:18:52 +0000 2020,1220954168057389056,1220954168057389056,Confused about how data science and data analy...,False,"[0, 198]","{'hashtags': [{'text': 'CareerKarma', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,1,0,False,False,en,False,NaN,NaN,NaN,NaN
3,Sat Jan 25 06:15:43 +0000 2020,1220953376189366272,1220953376189366272,Creating Robust Python Workflows: Learn to dev...,False,"[0, 164]","{'hashtags': [{'text': 'DataScience', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://twitter.com/DD_NaNa_"" rel=""no...",None,...,2,0,False,False,en,False,NaN,NaN,NaN,NaN
4,Sat Jan 25 06:11:32 +0000 2020,1220952323167440896,1220952323167440896,National Level Seminar on COMPUTATIONAL MATHEM...,False,"[0, 130]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'fr', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",None,...,2,0,False,False,fr,False,"{'media': [{'id': 1220952315739299840, 'id_str...",NaN,NaN,NaN
